In [1]:
import pandas as pd
import altair as alt
data = pd.read_csv("app-lab-data.csv") # This data is for two semesters (Fall 2021/ Spring 2022)

# Sun 3/31/2024 - the date that room reservations went live!

data.head()

/var/folders/k1/g9zgyyn94013vflzxszd6d840000gn/T/ipykernel_3731/4055143030.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,id,name,date,timeIn,timeOut,Duration,reason,classification,staff,comments,PID,hasPID,firstTime,heard_about_al_through
0,360,Charles Ma,2021-09-01,13:01:00,13:10:00,0:09:00,working,work/study/hang,NaN,NaN,730401800.0,True,True,Friend
1,362,Jonathan Segal,2021-09-01,14:22:00,14:43:00,0:21:00,checking stuff out,tour,NaN,NaN,730248182.0,True,True,Poster
2,361,Miranda Tran,2021-09-01,13:05:00,13:10:00,0:05:00,working,work/study/hang,NaN,NaN,730393052.0,True,True,Friend
3,363,Peter Close,2021-09-01,14:26:00,15:02:00,0:36:00,working,work/study/hang,NaN,NaN,730442455.0,True,False,NaN
4,364,Jonathan Segal,2021-09-02,12:30:00,13:41:00,1:11:00,app work,project/workshop/staff,NaN,NaN,730248182.0,True,False,NaN


In [2]:
# Create table for top 10 students in duration of time.  
total_time_per_user = data.groupby('id')['Duration'].sum().reset_index()

total_time_per_user['Duration'] = pd.to_timedelta(total_time_per_user['Duration']).dt.total_seconds() / 3600 

total_time_per_user_sorted = total_time_per_user.sort_values(by='Duration', ascending=False)

total_time_per_user_sorted.head(10)

,id,Duration
311,703,21.583333
312,704,21.500000
416,808,20.700000
508,901,20.400000
507,900,20.400000
509,902,20.333333
673,1130,17.533333
721,1178,6.816667
675,1132,6.716667
294,686,6.483333


In [3]:
# Visualization of the top users in each space.

chart = alt.Chart(total_time_per_user_sorted.head(10)).mark_bar().encode(
    y=alt.Y('id:O', title='User ID', sort="-x"),
    x=alt.X('Duration', title='Time (Hours)'),
    tooltip=alt.Tooltip('Duration')
).properties(
    title='Total Time per User (Hours)'
)

chart


alt.Chart(...)

In [4]:
# Do the leaderboard per day. 

# Convert string to TimeDelta Object.
data['date'] = pd.to_datetime(data['date'])

data['timeIn'] = pd.to_timedelta(data['timeIn'])

data['start'] = data['date'] + data['timeIn']

data['day_of_week'] = data['start'].dt.day_name()

data['civilian_time'] = data['start'].dt.strftime("%I %p").str.lstrip("0")

reservations_per_hour = data.groupby(["day_of_week", "civilian_time"]).size().reset_index(name="count")

reservations_per_hour.head(5)


,day_of_week,civilian_time,count
0,Friday,1 PM,11
1,Friday,10 AM,1
2,Friday,11 AM,2
3,Friday,12 PM,13
4,Friday,2 PM,12


In [5]:
# Load the chart

day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
charts = []
for day in day_order:
    day_data = reservations_per_hour[reservations_per_hour["day_of_week"] == day]

    chart = alt.Chart(day_data).mark_bar().encode( 
        x = alt.X("civilian_time:N", title="Hour of Day", sort=['10 AM', '11 AM', '12 PM', '1 PM', '2 PM', '3 PM', '4 PM', '5 PM']),
        y = alt.Y("count:Q", title=None),
        tooltip = ["count:Q"]
    ).properties(
        title=f'{day}',
        height=150
    )

    charts.append(chart)

final_chart = alt.hconcat(*charts)

final_chart

alt.HConcatChart(...)